# TensorflowでSVM
scikit-learn内にあるヒオウギアヤメのデータセットを使いTensorflowでSVMを実装していきます

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import datasets

まずは， データセットをロードする

In [ ]:
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals = np.array([1 if y==0 else -1 for y in iris.target])

学習，テストテータの分割

In [ ]:
x_vals_train,x_vals_test, y_vals_train, y_vals_test = train_test_split(x_vals, y_vals, test_size=0.3, random_state=0,)

識別関数$y=-\frac{a_2}{a_1}x+\frac{b}{a_1}$を学習する

In [ ]:
batch_size = 100
 
x_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
 
A = tf.Variable(tf.random_normal(shape=[2, 1]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))

In [ ]:
model_output = tf.subtract(tf.matmul(x_data, A), b)

In [ ]:
l2_norm = tf.constant([0.01])
alpha = tf.constant([0.01])
classification_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(model_output, y_target))))
loss = tf.add(classification_term, tf.multiply(alpha, l2_norm))

In [ ]:
prediction = tf.sign(model_output)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_target), tf.float32))

In [ ]:
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


In [ ]:
loss_vec = []
train_accuracy = []
test_accuracy = []
for i in range(1000):
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    rand_x = x_vals_train[rand_index]
    rand_y = np.transpose([y_vals_train[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
 
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
 
    train_acc_temp = sess.run(accuracy, feed_dict={
        x_data: x_vals_train,
        y_target: np.transpose([y_vals_train])})
    train_accuracy.append(train_acc_temp)
 
    test_acc_temp = sess.run(accuracy, feed_dict={
        x_data: x_vals_test,
        y_target: np.transpose([y_vals_test])})
    test_accuracy.append(test_acc_temp)
 
    if (i+1)%100==0:
        print('Step # {} A = {}, b = {}'.format(str(i+1), str(sess.run(A)), str(sess.run(b))))
        print('Loss = ' + str(temp_loss))

表示用に識別関数$y=-\frac{a_2}{a_1}x+\frac{b}{a_1}$を計算する

In [ ]:
[[a1], [a2]] = sess.run(A)
[[b]] = sess.run(b)
slope = -a2/a1
y_intercept = b/a1
x1_vals = [d[1] for d in x_vals]
 
best_fit = []
for i in x1_vals:
    best_fit.append(slope*i+y_intercept)

表示用にクラスごとに分離

In [ ]:
setosa_x = [d[1] for i,d in enumerate(x_vals) if y_vals[i]  == 1]
setosa_y = [d[0] for i,d in enumerate(x_vals) if y_vals[i]  == 1]
not_setosa_x = [d[1] for i,d in enumerate(x_vals) if y_vals[i] == -1]
not_setosa_y = [d[0] for i,d in enumerate(x_vals) if y_vals[i] == -1]

In [ ]:
plt.plot(setosa_x, setosa_y, 'o', label='I. setosa')
plt.plot(not_setosa_x, not_setosa_y, 'x', label='Non-setosa')
plt.plot(x1_vals, best_fit, 'r-', label='Liner Separator', linewidth=3)
plt.ylim([0, 10])
plt.legend(loc='lower right')
plt.title('Sepal Length vs Pedal Width')
plt.xlabel('Pedal Width')
plt.ylabel('Sepal Length')
plt.show()

http://challenge.no1s.biz/programming/python/371